## 1. 导入库

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 时间序列分析库
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose

# 评估指标
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 设置绘图样式
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (15, 6)

print("✅ 库导入完成")

## 2. 加载数据

In [ ]:
# 加载数据
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

# 转换时间戳
train_df['Timestamp'] = pd.to_datetime(train_df['Timestamp'])
test_df['Timestamp'] = pd.to_datetime(test_df['Timestamp'])

# 设置时间戳为索引
train_df.set_index('Timestamp', inplace=True)
test_df.set_index('Timestamp', inplace=True)

print(f"训练数据形状: {train_df.shape}")
print(f"测试数据形状: {test_df.shape}")
print(f"\n数据范围:")
print(f"训练集: {train_df.index.min()} 到 {train_df.index.max()}")
print(f"测试集: {test_df.index.min()} 到 {test_df.index.max()}")

train_df.head()

## 3. 数据探索和可视化

In [ ]:
# 查看Target列的时间序列图
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# 完整时间序列
axes[0].plot(train_df.index, train_df['Target'], linewidth=0.5)
axes[0].set_title('加密货币价格时间序列 - 完整数据', fontsize=14, fontweight='bold')
axes[0].set_xlabel('时间')
axes[0].set_ylabel('Target')
axes[0].grid(True, alpha=0.3)

# 最近1年的数据
recent_data = train_df.last('365D')
axes[1].plot(recent_data.index, recent_data['Target'], linewidth=0.8)
axes[1].set_title('最近1年数据', fontsize=12, fontweight='bold')
axes[1].set_xlabel('时间')
axes[1].set_ylabel('Target')
axes[1].grid(True, alpha=0.3)

# 最近30天的数据
recent_month = train_df.last('30D')
axes[2].plot(recent_month.index, recent_month['Target'], linewidth=1)
axes[2].set_title('最近30天数据', fontsize=12, fontweight='bold')
axes[2].set_xlabel('时间')
axes[2].set_ylabel('Target')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 统计描述
print("Target列统计信息:")
print(train_df['Target'].describe())
print(f"\n缺失值数量: {train_df['Target'].isna().sum()}")
print(f"零值数量: {(train_df['Target'] == 0).sum()}")

## 4. 平稳性检验 (Stationarity Test)

使用ADF检验(Augmented Dickey-Fuller Test)检查时间序列是否平稳。平稳性是ARIMA模型的前提条件。

In [ ]:
def adf_test(timeseries, name=''):
    """执行ADF检验"""
    print(f'ADF检验结果 {name}:')
    print('-' * 50)
    
    result = adfuller(timeseries.dropna(), autolag='AIC')
    
    print(f'ADF统计量: {result[0]:.6f}')
    print(f'p值: {result[1]:.6f}')
    print(f'使用的滞后阶数: {result[2]}')
    print(f'观测数: {result[3]}')
    print('临界值:')
    for key, value in result[4].items():
        print(f'  {key}: {value:.3f}')
    
    if result[1] <= 0.05:
        print("\n✅ 结论: 序列是平稳的 (拒绝原假设)")
        return True
    else:
        print("\n❌ 结论: 序列不平稳 (不能拒绝原假设)")
        return False

# 对原始数据进行ADF检验
is_stationary = adf_test(train_df['Target'], '原始数据')

## 5. 差分处理 (Differencing)

如果数据不平稳，需要进行差分处理。

In [ ]:
# 一阶差分
train_df['Target_diff1'] = train_df['Target'].diff()

# 二阶差分
train_df['Target_diff2'] = train_df['Target_diff1'].diff()

# 可视化差分效果
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# 原始数据
axes[0].plot(train_df.index, train_df['Target'], linewidth=0.5)
axes[0].set_title('原始数据', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Target')
axes[0].grid(True, alpha=0.3)

# 一阶差分
axes[1].plot(train_df.index, train_df['Target_diff1'], linewidth=0.5, color='orange')
axes[1].set_title('一阶差分', fontsize=12, fontweight='bold')
axes[1].set_ylabel('差分值')
axes[1].grid(True, alpha=0.3)

# 二阶差分
axes[2].plot(train_df.index, train_df['Target_diff2'], linewidth=0.5, color='green')
axes[2].set_title('二阶差分', fontsize=12, fontweight='bold')
axes[2].set_xlabel('时间')
axes[2].set_ylabel('差分值')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 对差分数据进行ADF检验
print('\n')
adf_test(train_df['Target_diff1'], '一阶差分')
print('\n')
adf_test(train_df['Target_diff2'], '二阶差分')

## 6. ACF和PACF分析

- **ACF (自相关函数)**: 衡量时间序列与其滞后值之间的相关性
- **PACF (偏自相关函数)**: 消除中间滞后影响后的相关性

### 如何解读ACF和PACF:

1. **AR(p)模型**: PACF在lag p后截断，ACF逐渐衰减
2. **MA(q)模型**: ACF在lag q后截断，PACF逐渐衰减
3. **ARMA(p,q)模型**: ACF和PACF都逐渐衰减

In [ ]:
# 使用平稳化的数据（一阶差分）进行ACF和PACF分析
stationary_data = train_df['Target_diff1'].dropna()

# 为了加快计算，可以使用最近的数据进行分析
# 使用最近10000个数据点
sample_size = min(10000, len(stationary_data))
stationary_sample = stationary_data.iloc[-sample_size:]

print(f"使用 {len(stationary_sample)} 个数据点进行ACF/PACF分析")

# 绘制ACF和PACF图
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# ACF图
plot_acf(stationary_sample, lags=40, ax=axes[0], alpha=0.05)
axes[0].set_title('自相关函数 (ACF)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('滞后阶数')
axes[0].set_ylabel('自相关系数')

# PACF图
plot_pacf(stationary_sample, lags=40, ax=axes[1], alpha=0.05, method='ywm')
axes[1].set_title('偏自相关函数 (PACF)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('滞后阶数')
axes[1].set_ylabel('偏自相关系数')

plt.tight_layout()
plt.show()

In [ ]:
# 计算ACF和PACF的数值
acf_values = acf(stationary_sample, nlags=40)
pacf_values = pacf(stationary_sample, nlags=40, method='ywm')

# 找出显著的滞后阶数（超出置信区间）
confidence_interval = 1.96 / np.sqrt(len(stationary_sample))

significant_acf_lags = np.where(np.abs(acf_values[1:]) > confidence_interval)[0] + 1
significant_pacf_lags = np.where(np.abs(pacf_values[1:]) > confidence_interval)[0] + 1

print("显著的ACF滞后阶数 (前10个):")
print(significant_acf_lags[:10])
print(f"\n显著的PACF滞后阶数 (前10个):")
print(significant_pacf_lags[:10])

# 根据ACF和PACF图推荐ARIMA参数
print("\n基于ACF和PACF分析的ARIMA参数建议:")
print("="*50)

# 简单的启发式规则
if len(significant_pacf_lags) > 0:
    p_suggest = min(significant_pacf_lags[:3].max() if len(significant_pacf_lags) >= 3 else significant_pacf_lags[0], 5)
else:
    p_suggest = 1

if len(significant_acf_lags) > 0:
    q_suggest = min(significant_acf_lags[:3].max() if len(significant_acf_lags) >= 3 else significant_acf_lags[0], 5)
else:
    q_suggest = 1

print(f"建议的AR阶数 (p): {p_suggest}")
print(f"建议的差分阶数 (d): 1 (基于ADF检验结果)")
print(f"建议的MA阶数 (q): {q_suggest}")
print(f"\n建议的ARIMA模型: ARIMA({p_suggest}, 1, {q_suggest})")

## 7. 季节性分解

检查数据是否有季节性模式，这将帮助我们决定是否使用SARIMA模型。

In [ ]:
# 使用最近的数据进行季节性分解（加快计算速度）
decompose_sample = train_df['Target'].iloc[-5000:]

# 季节性分解（假设周期为96，因为15分钟间隔，一天有96个时间点）
# 也可以尝试其他周期，如7*96（一周）
decomposition = seasonal_decompose(decompose_sample, model='additive', period=96)

# 绘制分解结果
fig, axes = plt.subplots(4, 1, figsize=(15, 12))

decomposition.observed.plot(ax=axes[0], title='原始数据')
axes[0].set_ylabel('观测值')

decomposition.trend.plot(ax=axes[1], title='趋势')
axes[1].set_ylabel('趋势')

decomposition.seasonal.plot(ax=axes[2], title='季节性')
axes[2].set_ylabel('季节性')

decomposition.resid.plot(ax=axes[3], title='残差')
axes[3].set_ylabel('残差')
axes[3].set_xlabel('时间')

plt.tight_layout()
plt.show()

## 8. ARIMA模型训练

使用多个ARIMA配置进行网格搜索，找到最佳参数。

In [ ]:
# 准备训练数据和验证数据
# 使用最后20%作为验证集
train_size = int(len(train_df) * 0.8)
train_data = train_df['Target'].iloc[:train_size]
val_data = train_df['Target'].iloc[train_size:]

print(f"训练集大小: {len(train_data)}")
print(f"验证集大小: {len(val_data)}")
print(f"训练集范围: {train_data.index.min()} 到 {train_data.index.max()}")
print(f"验证集范围: {val_data.index.min()} 到 {val_data.index.max()}")

In [ ]:
# ARIMA网格搜索
# 为了节省时间，我们测试几个常见的配置
p_values = [0, 1, 2]
d_values = [1]
q_values = [0, 1, 2]

best_aic = np.inf
best_order = None
best_model = None

results = []

print("开始ARIMA参数网格搜索...")
print("="*70)

for p in p_values:
    for d in d_values:
        for q in q_values:
            try:
                # 使用部分数据训练以加快速度
                sample_train = train_data.iloc[-5000:]
                
                model = ARIMA(sample_train, order=(p, d, q))
                fitted = model.fit()
                
                aic = fitted.aic
                bic = fitted.bic
                
                results.append({
                    'order': (p, d, q),
                    'AIC': aic,
                    'BIC': bic
                })
                
                print(f"ARIMA{(p,d,q)} - AIC: {aic:.2f}, BIC: {bic:.2f}")
                
                if aic < best_aic:
                    best_aic = aic
                    best_order = (p, d, q)
                    best_model = fitted
                    
            except Exception as e:
                print(f"ARIMA{(p,d,q)} - 失败: {str(e)}")
                continue

print("\n" + "="*70)
print(f"最佳ARIMA模型: ARIMA{best_order}")
print(f"最佳AIC: {best_aic:.2f}")
print("="*70)

In [ ]:
# 显示所有结果
results_df = pd.DataFrame(results)
results_df = results_df.sort_values('AIC')
print("\n所有ARIMA模型结果（按AIC排序）:")
print(results_df.to_string(index=False))

## 9. 使用最佳模型进行预测

In [ ]:
# 使用最佳参数在完整训练集上重新训练
print(f"使用ARIMA{best_order}在完整训练集上训练...")
print("注意: 这可能需要较长时间...")

final_model = ARIMA(train_data, order=best_order)
final_fitted = final_model.fit()

print("\n模型摘要:")
print(final_fitted.summary())

In [ ]:
# 在验证集上进行预测
val_predictions = final_fitted.forecast(steps=len(val_data))

# 计算评估指标
rmse = np.sqrt(mean_squared_error(val_data, val_predictions))
mae = mean_absolute_error(val_data, val_predictions)
r2 = r2_score(val_data, val_predictions)

print(f"验证集评估结果:")
print(f"  RMSE: {rmse:.6f}")
print(f"  MAE:  {mae:.6f}")
print(f"  R²:   {r2:.6f}")

In [ ]:
# 可视化验证集预测结果
plt.figure(figsize=(15, 6))

# 只显示最后1000个点，以便更清晰地看到细节
display_size = min(1000, len(val_data))

plt.plot(val_data.index[-display_size:], val_data.iloc[-display_size:], 
         label='实际值', linewidth=1.5, alpha=0.7)
plt.plot(val_data.index[-display_size:], val_predictions[-display_size:], 
         label='预测值', linewidth=1.5, alpha=0.7)

plt.title(f'ARIMA{best_order} 验证集预测结果 (最后{display_size}个点)', fontsize=14, fontweight='bold')
plt.xlabel('时间')
plt.ylabel('Target')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 残差分析
residuals = val_data.values - val_predictions

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 残差时间序列
axes[0, 0].plot(val_data.index, residuals, linewidth=0.5)
axes[0, 0].axhline(y=0, color='r', linestyle='--', alpha=0.7)
axes[0, 0].set_title('残差时间序列')
axes[0, 0].set_xlabel('时间')
axes[0, 0].set_ylabel('残差')
axes[0, 0].grid(True, alpha=0.3)

# 残差分布
axes[0, 1].hist(residuals, bins=50, edgecolor='black')
axes[0, 1].set_title('残差分布')
axes[0, 1].set_xlabel('残差')
axes[0, 1].set_ylabel('频数')
axes[0, 1].grid(True, alpha=0.3)

# Q-Q图
from scipy import stats
stats.probplot(residuals, dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q图')
axes[1, 0].grid(True, alpha=0.3)

# 残差ACF
plot_acf(residuals, lags=40, ax=axes[1, 1], alpha=0.05)
axes[1, 1].set_title('残差ACF')

plt.tight_layout()
plt.show()

## 10. 对测试集进行预测

In [ ]:
# 在完整训练集上重新训练最终模型
print("在完整训练集上训练最终模型...")
final_model_full = ARIMA(train_df['Target'], order=best_order)
final_fitted_full = final_model_full.fit()

# 预测测试集
test_steps = len(test_df)
test_predictions = final_fitted_full.forecast(steps=test_steps)

print(f"\n测试集预测完成!")
print(f"预测数量: {len(test_predictions)}")
print(f"预测值范围: [{test_predictions.min():.6f}, {test_predictions.max():.6f}]")
print(f"预测值均值: {test_predictions.mean():.6f}")
print(f"预测值标准差: {test_predictions.std():.6f}")

In [ ]:
# 可视化测试集预测
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# 训练集最后部分和预测
last_train_points = 500
axes[0].plot(train_df.index[-last_train_points:], train_df['Target'].iloc[-last_train_points:], 
             label='训练集(最后500点)', linewidth=1)
axes[0].plot(test_df.index, test_predictions, label='测试集预测', linewidth=1, color='orange')
axes[0].set_title('训练集和测试集预测', fontsize=14, fontweight='bold')
axes[0].set_xlabel('时间')
axes[0].set_ylabel('Target')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 只显示测试集预测
axes[1].plot(test_df.index, test_predictions, linewidth=1, color='orange')
axes[1].set_title('测试集预测结果', fontsize=14, fontweight='bold')
axes[1].set_xlabel('时间')
axes[1].set_ylabel('预测值')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 11. 生成提交文件

In [ ]:
# 创建提交文件
submission_df = pd.DataFrame({
    'Timestamp': test_df.index,
    'Target': test_predictions
})

# 重置索引
submission_df.reset_index(drop=True, inplace=True)

# 保存提交文件
submission_dir = Path('../submissions')
submission_dir.mkdir(exist_ok=True)

submission_file = submission_dir / 'arima_submission.csv'
submission_df.to_csv(submission_file, index=False)

print(f"✅ 提交文件已保存: {submission_file}")
print(f"\n提交文件形状: {submission_df.shape}")
print("\n提交文件预览:")
submission_df.head(10)

In [ ]:
# 提交文件统计
print("提交文件统计:")
print(submission_df['Target'].describe())

## 12. 模型诊断

检查模型的残差是否满足白噪声假设。

In [ ]:
# 绘制模型诊断图
final_fitted_full.plot_diagnostics(figsize=(15, 12))
plt.tight_layout()
plt.show()

## 总结

### 使用的方法:
1. **平稳性检验**: 使用ADF检验确定数据的平稳性
2. **差分**: 通过差分使数据平稳
3. **ACF/PACF分析**: 确定ARIMA模型的参数(p, d, q)
4. **网格搜索**: 通过AIC/BIC选择最佳参数
5. **ARIMA建模**: 训练最佳ARIMA模型
6. **残差分析**: 验证模型的有效性

### 下一步改进:
- 尝试SARIMA模型（包含季节性）
- 使用外生变量(ARIMAX)
- 尝试更多的参数组合
- 考虑使用Prophet或其他现代时间序列模型
- 集成多个时间序列模型